# Setup

In [1]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


kj/filesystem-disk-unix.c++:1734: warning: PWD environment variable doesn't match current directory; pwd = /


# Connecting to the Exchange

In [2]:
exchange = Exchange()
_ = exchange.connect()

2024-03-02 15:12:04,448 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Getting market information

In [3]:
# Load all instruments (products) available on the exchange
instruments = exchange.get_instruments()
instruments

{'CSCO': Instrument(instrument_id=CSCO, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=1.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=CSCO, paused=False, expired=False),
 'PFE': Instrument(instrument_id=PFE, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=1.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=PFE, paused=False, expired=False),
 'SAN': Instrument(instrument_id=SAN, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=1.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=SAN, paused=False, expired=False),
 'ING': Instrument(instrument_id=ING, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=1.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=ING, paused=False, expired=False),
 'NVDA': Instrument(instrument_id=NVDA, tick_size=0.1, price_change_limit=PriceChange

In [4]:
# Define a variable storing the instrument_id of the product we are interested in
instrument_id = 'DEMO'

### Order Book

In [5]:
# Load current limit order book for the instrument
exchange.get_last_price_book(instrument_id)

### Public tradeticks

In [6]:
# Load and store a list of all public tradeticks since the instantiation of the Exchange (upto a max limit)
# These are trades between any participant, you may or may not have participated in these
trade_tick_history = exchange.get_trade_tick_history(instrument_id)

# Display last 5
trade_tick_history[-5:]

[]

In [7]:
# Poll all public tradeticks since the last time this method was called
exchange.poll_new_trade_ticks(instrument_id)

[]

### Private trades

In [8]:
# Load and store a list of all private trades you participated in since the instantiation of the Exchange (upto a max limit)
trade_history = exchange.get_trade_history(instrument_id)

# Display last 5
trade_history[-5:]

[]

In [9]:
# Poll all private trades since the last time this method was called
exchange.poll_new_trades(instrument_id)

[]

### Position & PnL

In [10]:
# Load current positions in all instruments
exchange.get_positions()

{'NVDA': -10, 'ING': 0, 'SAN': 0, 'PFE': 10, 'CSCO': 10}

In [11]:
# Load current positions in all instruments, including cash component (how much did we invest or gain by trading each instrument)
exchange.get_positions_and_cash()

{'NVDA': {'volume': -10, 'cash': 528.9999999999998},
 'ING': {'volume': 0, 'cash': 91.99999999999903},
 'SAN': {'volume': 0, 'cash': 94.00000000000068},
 'PFE': {'volume': 10, 'cash': -672.9999999999992},
 'CSCO': {'volume': 10, 'cash': -343.0000000000005}}

In [12]:
# Current PnL: sum of all cash components and value of the positions (evaluated against last-traded price) 
exchange.get_pnl()

503.9999999999998

# Orders

### Inserting

In [13]:
# Insert bid LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
# The returned value is a 'InsertOrderResponse' which, if successfull, contains the order_id of the order
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='limit')

2024-03-02 15:12:20,277 [client    ] [Thread-4 (_thread_entry_point)] Forcing a disconnect due to an error: /home/ubuntu/optibook/libs/server/server.h:224: failed: expected it != mInsts.end(); Instrument does not exist
stack: 55a4cfbda4b3 55a4cfc12a7a 55a4cfc2fc4b 55a4cfc2fd0a 55a4cfc48909 55a4cfc50436 55a4cfcc83e1 55a4cfd23dda 55a4cfcc6ce6 55a4cfccbeb2 55a4cfcc7c84 55a4cfcce9d2 55a4cfbcdc45 55a4cfbcc849 7f4ce8868082 55a4cfbccefd.


KjException: (remote):0: failed: remote exception: expected it != mInsts.end(); Instrument does not exist
stack: 7f905f48d640 7f905f47fc70 7f905f322f90 7f905f324750

In [14]:
# Insert ask LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='limit')

AssertionError: Cannot call function until connected. Call connect() first

In [15]:
# Insert bid IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='ioc')

AssertionError: Cannot call function until connected. Call connect() first

In [16]:
# Insert ask IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='ioc')

AssertionError: Cannot call function until connected. Call connect() first

### Deleting & amending

In [17]:
# Load a list of own outstanding/resting orders
exchange.get_outstanding_orders(instrument_id)

AssertionError: Cannot call function until connected. Call connect() first

In [18]:
# Delete an order by order_id, the returned DeleteOrderRequest contains a boolean showing whether the delete was succesful
exchange.delete_order(instrument_id, order_id=1234)

AssertionError: Cannot call function until connected. Call connect() first

In [19]:
# Delete all outstanding orders for an instrument
exchange.delete_orders(instrument_id)

AssertionError: Cannot call function until connected. Call connect() first

In [20]:
# Change (amend) the volume of an existing resting order by order_id, the returned AmendOrderResponse contains a boolean showing whether the amend was succesful
exchange.amend_order(instrument_id, order_id=1234, volume=30)

AssertionError: Cannot call function until connected. Call connect() first

# Code Snippets

Combining a few of the exchange interactions above, we can write a code snippet to aggressively trade out of all currently held positions using IOC orders. 

That means selling all products you are long, and buying back all you are short. 

This is useful for starting with a clean slate, but doing so will come at a potentially large cost. 

You are not guaranteed to end up with a zero position afterwards, only that volume which is also available to trade in the market will be bought/sold.

In [21]:
MIN_SELLING_PRICE = 0.10
MAX_BUYING_PRICE = 100000.00

positions = exchange.get_positions()
pnl = exchange.get_pnl()

print(f'Positions before: {positions}')
print(f'\nPnL before: {pnl:.2f}')

print(f'\nTrading out of positions')
for iid, pos in positions.items():
    if pos > 0:
        print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
        exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
    elif pos < 0:
        print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
        exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
    else:
        print(f'-- No initial position in {iid}, skipping..')
    
    time.sleep(0.10)

time.sleep(1.0)

positions = exchange.get_positions()
pnl = exchange.get_pnl()
print(f'\nPositions after: {positions}')
print(f'\nPnL after: {pnl:.2f}')

AssertionError: Cannot call function until connected. Call connect() first